# Download the images from the tweets and push to s3 for scalable CDN
(Reference: https://github.com/INFO-4602-5602/final-project-tweetsonamap/issues/2)

In [1]:
import pymongo, json, pprint, urllib.request, os.path, sys
from html.parser import HTMLParser

Connect to the DB

    client = pymongo.MongoClient('mongodb://epic-analytics.cs.colorado.edu')
    db = client.matthew
    print(db.collection_names())
    tweets = db.tweets
    print(tweets.count(), "tweets")

In [2]:
client = pymongo.MongoClient('localhost')
tweets = client['matthew'].tweets

In [3]:
tweets.count()

13830253

###  Image Handlers

In [3]:
err_log = open('scraping_errors_3.log','w')

class InstagramHandler(HTMLParser):
    """
        Custom function to handle scraping images from Instagram... totally violates TOS, I think.
    """
    def __init__(self, tweet_id):
        self.image_url = None
        self.tweet_id = tweet_id
        self.directory = "/data/infovis/instagram"
        super().__init__()
        
    def image_needed(self):
        if os.path.exists(self.directory+"/"+self.tweet_id+".jpg"):
            err_log.write("File Exists: "+self.tweet_id+"\n")
            return False
        else:
            return True

    def handle_starttag(self, tag, attrs):
        if tag=='meta':
            if attrs[0][1]=='og:image':
                self.image_url = attrs[1][1]
                try:
                    req = urllib.request.urlopen(self.image_url)
                    with open(self.directory+"/"+self.tweet_id+".jpg",'wb') as pic:
                        pic.write(req.read())
                except:
                    err_log.write("Error, image not found: "+self.tweet_id+"\n")

In [4]:
cursor = tweets.find({'geo' : {"$exists":True}})

In [ ]:
idx = 0;
for t in cursor:
    tweet_id = t['id'].split(":")[2]
    for url in t['twitter_entities']['urls']:
        sys.stderr.write("\r {0}: {1}".format(idx,tweet_id))
        idx+=1;

        #Instagram Case
        if url['expanded_url'].startswith("https://www.instagram.com"):            
            parser = InstagramHandler(tweet_id)
            if parser.image_needed():
                try:
                    req = urllib.request.urlopen(url['expanded_url'])
                    parser.feed(req.read().decode('utf-8'))
                except:
                    err_log.write("Error, page not found: "+tweet_id + "\n")
err_log.close()

 0: 779833573184536576-c:11: DeprecationWarning: The value of convert_charrefs will become True in 3.5. You are encouraged to set the value explicitly.
 8911: 783614745026830336

## Get images from geotagged tweets

In [4]:
geo_image_tweets = list(tweets.find({
    "verb":"post", # original tweet
    "twitter_extended_entities.media.0":{"$exists":True}, # at least 1 media entity
    "twitter_extended_entities.media": {"$all":[{"$elemMatch": { "type": "photo" }}]}, # all media entities are photos
    "location.geo.coordinates":{"$exists":True}
}))

In [18]:
len(geo_image_tweets)

23789

In [27]:
class TwitterHandler(HTMLParser):
    """
        Custom function to handle downloading tweet images.
    """
    def __init__(self, tweet_id):
        self.image_url = None
        self.tweet_id = tweet_id
        self.directory = "/data/infovis/twitter"
        super().__init__()
        
    def image_needed(self):
        if os.path.exists(self.directory+"/"+self.tweet_id+".jpg"):
            err_log.write("File Exists: "+self.tweet_id+"\n")
            return False
        else:
            return True

In [29]:
err_log = open('scraping_errors_twitter.log','w')

idx = 0;
for t in geo_image_tweets[:10]:
    tweet_id = t['id'].split(":")[2]
    img_url = t['twitter_extended_entities']['media'][0]['media_url_https']
    print(img_url)
    sys.stderr.write("\r {0}: {1}".format(idx,tweet_id))
    idx+=1;

    parser = TwitterHandler(tweet_id)
    if parser.image_needed():
        try:
            req = urllib.request.urlopen(img_url)
            parser.feed(req.read().decode('utf-8'))
        except:
            err_log.write("Error, page not found: "+tweet_id + "\n")
err_log.close()

https://pbs.twimg.com/media/CtKGGh_XgAAZiZB.jpg
https://pbs.twimg.com/media/CtKGvDIUsAAO_ku.jpg
https://pbs.twimg.com/media/CtKHV4yWIAABbJY.jpg
https://pbs.twimg.com/media/CtKKt44VYAAIs5w.jpg
https://pbs.twimg.com/media/CtKPWsrVMAAz1za.jpg
https://pbs.twimg.com/media/B2hfaeSIMAEJq5d.jpg
https://pbs.twimg.com/media/BuR8ET2IcAEY1aR.jpg
https://pbs.twimg.com/media/CtKUcX-VMAA9ENY.jpg
https://pbs.twimg.com/media/CtKWEuiXEAArmSP.jpg
https://pbs.twimg.com/media/CtKW9zhUAAAuzfa.jpg


 0: 779833156773871616-c:9: DeprecationWarning: The value of convert_charrefs will become True in 3.5. You are encouraged to set the value explicitly.
 9: 779851721065402368

In [16]:
# i=0
# for tweet in geo_image_tweets:
#     if i<100:
#         i += 1

#         img_url = tweet['twitter_extended_entities']['media'][0]['media_url_https']
#         img_name = img_url.split("/")[-1]
#         print(img_url)

#         # save image locally
#         try:
#             urllib.request.urlretrieve(img_url, "/data/infovis/twitter/"+img_name)
#         except:
#             e = sys.exc_info()[0]
#             print(e, img_url)

https://pbs.twimg.com/media/CtKGGh_XgAAZiZB.jpg
<class 'FileNotFoundError'> https://pbs.twimg.com/media/CtKGGh_XgAAZiZB.jpg
https://pbs.twimg.com/media/CtKGvDIUsAAO_ku.jpg
<class 'FileNotFoundError'> https://pbs.twimg.com/media/CtKGvDIUsAAO_ku.jpg
https://pbs.twimg.com/media/CtKHV4yWIAABbJY.jpg
<class 'FileNotFoundError'> https://pbs.twimg.com/media/CtKHV4yWIAABbJY.jpg
https://pbs.twimg.com/media/CtKKt44VYAAIs5w.jpg
<class 'FileNotFoundError'> https://pbs.twimg.com/media/CtKKt44VYAAIs5w.jpg
https://pbs.twimg.com/media/CtKPWsrVMAAz1za.jpg
<class 'FileNotFoundError'> https://pbs.twimg.com/media/CtKPWsrVMAAz1za.jpg
https://pbs.twimg.com/media/B2hfaeSIMAEJq5d.jpg
<class 'FileNotFoundError'> https://pbs.twimg.com/media/B2hfaeSIMAEJq5d.jpg
https://pbs.twimg.com/media/BuR8ET2IcAEY1aR.jpg
<class 'FileNotFoundError'> https://pbs.twimg.com/media/BuR8ET2IcAEY1aR.jpg
https://pbs.twimg.com/media/CtKUcX-VMAA9ENY.jpg
<class 'FileNotFoundError'> https://pbs.twimg.com/media/CtKUcX-VMAA9ENY.jpg
https://